In [23]:
import os 
import datetime
import numpy as np 
import pandas as pd 
import vectorbt as vbt 

In [24]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 3)

In [82]:
btc_price = vbt.YFData.download(
    ["BTC-USD", "ETH-USD"],
    interval = "1m",
    start = start_date,
    end = end_date,    
    missing_index="drop").get("Close")


c:\Users\ktsar\anaconda3\envs\pytorch\Lib\site-packages\vectorbt\data\base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



In [26]:
print(type(btc_price))

<class 'pandas.core.frame.DataFrame'>


In [83]:
btc_price

symbol,BTC-USD,ETH-USD
Datetime,,
2024-05-07 13:19:00+00:00,63708.148438,3080.911133
2024-05-07 13:20:00+00:00,63679.140625,3080.206055
2024-05-07 13:21:00+00:00,63673.796875,3080.205322
2024-05-07 13:22:00+00:00,63657.199219,3079.444336
2024-05-07 13:23:00+00:00,63631.050781,3078.032959
...,...,...
2024-05-10 13:13:00+00:00,62994.140625,3027.322021
2024-05-10 13:15:00+00:00,62989.457031,3026.509521
2024-05-10 13:16:00+00:00,62995.140625,3026.660645


In [28]:
rsi = vbt.RSI.run(btc_price, window = 14)

print(rsi.rsi)

rsi_window                        14           
symbol                       BTC-USD    ETH-USD
Datetime                                       
2024-05-07 13:19:00+00:00        NaN        NaN
2024-05-07 13:20:00+00:00        NaN        NaN
2024-05-07 13:21:00+00:00        NaN        NaN
2024-05-07 13:22:00+00:00        NaN        NaN
2024-05-07 13:23:00+00:00        NaN        NaN
...                              ...        ...
2024-05-10 13:09:00+00:00  33.321036  47.232755
2024-05-10 13:10:00+00:00  37.640203  50.597323
2024-05-10 13:11:00+00:00  45.490275  56.105769
2024-05-10 13:12:00+00:00  49.351740  52.455906
2024-05-10 13:13:00+00:00  51.068653  53.004440

[3693 rows x 2 columns]


In [29]:
entries = rsi.rsi_crossed_below(30)
exits = rsi.rsi_crossed_above(70)
#print(entries.to_string())

In [52]:
pf = vbt.Portfolio.from_signals(btc_price['BTC-USD'], entries, exits,freq="1m" )
print(pf.stats())

Start                         2024-05-07 13:19:00+00:00
End                           2024-05-10 13:13:00+00:00
Period                                  2 days 13:33:00
Start Value                                       100.0
End Value                                    100.064303
Total Return [%]                               0.064303
Benchmark Return [%]                          -1.120748
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               2.666773
Max Drawdown Duration         1 days 17:20:15.833333333
Total Trades                                  37.472222
Total Closed Trades                           36.638889
Total Open Trades                              0.833333
Open Trade PnL                                -0.161214
Win Rate [%]                                  61.958508
Best Trade [%]                                 0.731915
Worst Trade [%]                               -1

C:\Users\ktsar\AppData\Local\Temp\ipykernel_29356\4067585139.py:2: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000024148977420>. Pass column to select a single column/group.



In [53]:
print(pf.total_return())

comb_rsi_window  comb_ma_window  comb_entry  comb_exit  symbol 
14               21              30          70         BTC-USD    0.005494
                                                        ETH-USD    0.004112
                                             60         BTC-USD    0.012767
                                                        ETH-USD   -0.007936
                                 40          70         BTC-USD   -0.000090
                                                                     ...   
21               100             30          60         ETH-USD    0.001221
                                 40          70         BTC-USD   -0.007396
                                                        ETH-USD   -0.021380
                                             60         BTC-USD   -0.006405
                                                        ETH-USD   -0.004072
Name: total_return, Length: 72, dtype: float64


In [11]:
pf.plot().show()

### CREATING CUSTOM INDICATORS ###

In [54]:
print(btc_price)

symbol                          BTC-USD      ETH-USD
Datetime                                            
2024-05-07 13:19:00+00:00  63708.148438  3080.911133
2024-05-07 13:20:00+00:00  63679.140625  3080.206055
2024-05-07 13:21:00+00:00  63673.796875  3080.205322
2024-05-07 13:22:00+00:00  63657.199219  3079.444336
2024-05-07 13:23:00+00:00  63631.050781  3078.032959
...                                 ...          ...
2024-05-10 13:09:00+00:00  62959.609375  3027.390869
2024-05-10 13:10:00+00:00  62959.183594  3026.742188
2024-05-10 13:11:00+00:00  62965.695312  3026.784180
2024-05-10 13:12:00+00:00  62965.695312  3026.784180
2024-05-10 13:13:00+00:00  62994.140625  3027.322021

[3693 rows x 2 columns]


In [55]:
def custom_indicator(close, rsi_window = 14, ma_window = 50, entry = 30, exit = 70):
    rsi = vbt.RSI.run(close, window= rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > exit, -1, 0)
    trend = np.where((rsi < entry) & (close < ma), 1, trend)
    
    return trend

ind = vbt.IndicatorFactory(class_name=  "Combination",
                           short_name=  "comb",
                           input_names = ["close"],
                           param_names = ['rsi_window', 'ma_window', 'entry', 'exit'],
                           output_names=['value']).from_apply_func(custom_indicator, 
                                                                   rsi_window = 14, 
                                                                   ma_window = 50,
                                                                   entry = 30,
                                                                   exit = 70,
                                                                   keep_pd = True)

res = ind.run(btc_price, 
              rsi_window = [14,35,21], 
              ma_window = [21,50, 100],
              entry = [30,40],
              exit = [70,60],
              param_product = True)

print(res.value)

entries = res.value == 1.0
exits = res.value == -1.0

comb_rsi_window                14                                          \
comb_ma_window                21                                            
comb_entry                     30                              40           
comb_exit                      70              60              70           
symbol                    BTC-USD ETH-USD BTC-USD ETH-USD BTC-USD ETH-USD   
Datetime                                                                    
2024-05-07 13:19:00+00:00       0       0       0       0       0       0   
2024-05-07 13:20:00+00:00       0       0       0       0       0       0   
2024-05-07 13:21:00+00:00       0       0       0       0       0       0   
2024-05-07 13:22:00+00:00       0       0       0       0       0       0   
2024-05-07 13:23:00+00:00       0       0       0       0       0       0   
...                           ...     ...     ...     ...     ...     ...   
2024-05-10 13:09:00+00:00       0       0       0       0       1       0   

In [39]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, freq= "1m")

print(pf.stats())

Start                         2024-05-07 13:19:00+00:00
End                           2024-05-10 13:13:00+00:00
Period                                  2 days 13:33:00
Start Value                                       100.0
End Value                                    100.539331
Total Return [%]                               0.539331
Benchmark Return [%]                           -1.43007
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               2.502142
Max Drawdown Duration            1 days 13:06:32.500000
Total Trades                                  37.472222
Total Closed Trades                           36.638889
Total Open Trades                              0.833333
Open Trade PnL                                -0.083127
Win Rate [%]                                  64.255932
Best Trade [%]                                 0.749323
Worst Trade [%]                               -1

C:\Users\ktsar\AppData\Local\Temp\ipykernel_29356\2660161974.py:3: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000024148977420>. Pass column to select a single column/group.



### HYPERPARAMETER OPTIMIZATION ###

You do not run this as it will aggregate the columns and you will not get a clear picture

pf = vbt.Portfolio.from_signals(btc_price, entries, exits, freq= "1m")

print(pf.stats())


Provide list of values or numpy arrays to create to the input functions and VectorBT will automatically backtests them.  

In [73]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, freq= "1m")

print(pf.total_return())

comb_rsi_window  comb_ma_window  comb_entry  comb_exit  symbol 
14               21              30          70         BTC-USD    0.005494
                                                        ETH-USD    0.006732
                                             60         BTC-USD    0.012767
                                                        ETH-USD   -0.007844
                                 40          70         BTC-USD   -0.000090
                                                                     ...   
21               100             30          60         ETH-USD    0.011811
                                 40          70         BTC-USD   -0.007396
                                                        ETH-USD   -0.000704
                                             60         BTC-USD   -0.006405
                                                        ETH-USD    0.001799
Name: total_return, Length: 72, dtype: float64


In [74]:
returns = pf.total_return()

"""returns = returns[returns.index.isin(['ETH-USD'], level = "symbol")]"""
#returns = returns.groupby(level = ['comb_exit', 'comb_entry']).mean()


print(returns.to_string())
print(returns.max())
print(returns.idxmax())

comb_rsi_window  comb_ma_window  comb_entry  comb_exit  symbol 
14               21              30          70         BTC-USD    0.005494
                                                        ETH-USD    0.006732
                                             60         BTC-USD    0.012767
                                                        ETH-USD   -0.007844
                                 40          70         BTC-USD   -0.000090
                                                        ETH-USD    0.016342
                                             60         BTC-USD    0.007176
                                                        ETH-USD   -0.005045
                 50              30          70         BTC-USD    0.003217
                                                        ETH-USD    0.004980
                                             60         BTC-USD    0.010422
                                                        ETH-USD   -0.008198
                        

In [75]:
#"comb_rsi_window ,comb_ma_window "

fig = returns.vbt.heatmap(x_level = "comb_exit",
                          y_level = "comb_entry",
                          slider_level = "symbol")

fig.show()

### OPTIMIZATION TECHNIQUES ###

In [85]:
import talib ### low level C cide
from numba import njit

In [87]:
RSI = vbt.IndicatorFactory.from_talib("RSI")

@njit # Numba decorators to speed up code 
def produce_signal(rsi, entry, exit):
    trend = np.where(rsi > exit, -1, 0)
    trend = np.where((rsi < entry), 1, trend)
    return trend

def custom_indicator(close, rsi_window = 14, entry = 30, exit = 70):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    return produce_signal(rsi, entry, exit)

ind = vbt.IndicatorFactory(class_name=  "Combination",
                           short_name=  "comb",
                           input_names = ["close"],
                           param_names = ['rsi_window', 'entry', 'exit'],
                           output_names=['value']).from_apply_func(custom_indicator, 
                                                                   rsi_window = 14, 
                                                                   entry = 30,
                                                                   exit = 70
                                                                   )

res = ind.run(btc_price, 
              rsi_window = np.arange(10,40,step = 3, dtype = int), 
              entry = np.arange(10,40, step = 2, dtype = int),
              exit = np.arange(60, 85, step = 2, dtype = int),
              param_product = True)

print(res.value)

entries = res.value == 1.0
exits = res.value == -1.0

comb_rsi_window                10                                          \
comb_entry                     10                                           
comb_exit                      60              62              64           
symbol                    BTC-USD ETH-USD BTC-USD ETH-USD BTC-USD ETH-USD   
Datetime                                                                    
2024-05-07 13:19:00+00:00       0       0       0       0       0       0   
2024-05-07 13:20:00+00:00       0       0       0       0       0       0   
2024-05-07 13:21:00+00:00       0       0       0       0       0       0   
2024-05-07 13:22:00+00:00       0       0       0       0       0       0   
2024-05-07 13:23:00+00:00       0       0       0       0       0       0   
...                           ...     ...     ...     ...     ...     ...   
2024-05-10 13:13:00+00:00       0       0       0       0       0       0   
2024-05-10 13:15:00+00:00       0       0       0       0       0       0   